# vaderSentiment

## Imports

In [1]:
import pandas as pd

from sqlalchemy import create_engine

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## Load data

In [107]:
en = create_engine('mysql+mysqlconnector://duinmat:$Mo/qdGSnX$MEi@oege.ie.hva.nl/zduinmat')

In [112]:
dbConnection = en.connect()

In [113]:
df = pd.read_sql("select * from Tweets_unedited", dbConnection)

In [ ]:
#df.info()

## Sentiment anylsis

Label: 1 - pos , 0 - neu , -1 - neg , 3 - error

In [116]:
analyzer = SentimentIntensityAnalyzer()

In [117]:
sentiment = []

In [118]:
error = 0
for idx,row in enumerate(df.iterrows()):
    try:
        vs = analyzer.polarity_scores(row[1]["content"])
        
        if vs["compound"] >= 0.05:
            sentiment.append((idx,1,"pos"))
        elif vs["compound"] > -0.05 and vs["compound"] < 0.05:
            sentiment.append((idx,0,"neu"))
        elif vs["compound"] <= -0.05:
            sentiment.append((idx,-1,"neg"))
        else:
            sentiment.append((idx,3,"error"))
    except Exception as e:
        sentiment.append((idx,3,"error"))
        error += 1
        print("*"*50)
        print("Index:" , idx)
        print("Error: ", e)
        print("Contend", row[1]["content"])
        print("*"*50)
print("Errors: " ,error)

Errors:  0


In [119]:
df_sentiment = pd.DataFrame(sentiment, columns=["Index","Sentiment_number","Sentiment_explain"])

In [120]:
df["vaderSentiment_number"] = df_sentiment["Sentiment_number"]
df["vaderSentiment_explain"] = df_sentiment["Sentiment_explain"]

## Save data in MySQL Database

In [24]:
en = create_engine('mysql+mysqlconnector://duinmat:$Mo/qdGSnX$MEi@oege.ie.hva.nl/zduinmat')

In [123]:
df.to_sql("Tweets_unedited_with_vaderSentiment",con = en,if_exists='replace',chunksize=10000)